In [1]:
pip install python-docx 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.6 MB 5.3 MB/s 
  Created wheel for python-docx: filename=python_docx-0.8.11-py3-none-any.whl size=184507 sha256=83715b60f7d59e5ea4590b5c4ed831d1893bf913825cb29068b1bc197736b26c
  Stored in directory: /root/.cache/pip/wheels/f6/6f/b9/d798122a8b55b74ad30b5f52b01482169b445fbb84a11797a6
Successfully built python-docx


In [2]:
pip install pyPDF2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 200 kB 5.6 MB/s 


In [3]:
pip install html2text

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


###Just imported some packages that are used for ETL

In [85]:
import pandas as pd
import numpy as np
import pathlib
import os
import re
import glob 
import docx
from docx import Document
import PyPDF2
from PyPDF2 import PdfReader
import html2text
from bs4 import BeautifulSoup
import pickle

*auto_load* functionalities:
1.   Accepts the input folder path.
2.   A For loop feeds in different data extensions ['/*.doc*','/*.pdf','/*.html','/*.txt']
3.   Glob fetches the files and appends them into the *input_files* list.
4.   *input_files* list contains the paths of all the detected files.



In [5]:
def auto_load(folder_path):
    input_files = []
    data_types = ['/*.doc*','/*.pdf','/*.html','/*.txt']
    for i in data_types:
        temp_input_files = glob.glob(folder_path+str(i))
        input_files.extend(temp_input_files)
    return input_files

Explanation

1.   The first "for loop will" itrate the file paths
2.   "If condition" checks the file extensions.
3.   In the if blocks, the predefined functions transforms 
     the data types to text format.
4.   All the files are then appended in a dict where 
"keys" stores the file names and "values" stores the respective text data.



In [84]:
def doc_to_text(file_path):
    main_doc_list = []
    text_1= ''
    out_dict= {}
    for doc in file_path:
        file_extension = pathlib.Path(doc).suffix
        filename = os.path.basename(doc)
        if file_extension == '.docx':
            word_doc = docx.Document(doc) 
            for words in word_doc.paragraphs:
                text_1 += words.text 
            main_doc_list.append(text_1)
            text_1= ''
            for document_no in enumerate(main_doc_list):
                out_dict.update({filename:main_doc_list[document_no[0]]})

        if file_extension == '.pdf':
            reader = PdfReader(doc)
            number_of_pages = len(reader.pages)
            for page_no in range(number_of_pages):
                page = reader.pages[page_no]
                text_1 += page.extract_text()
            main_doc_list.append(text_1)
            text_1= ''
            for document_no in enumerate(main_doc_list):
                out_dict.update({filename:main_doc_list[document_no[0]]})
 
        if file_extension == '.html':
            file_handle = open(doc, 'r')
            h = html2text.HTML2Text()
            h.igonre_links= True
            html_data = file_handle.read()
            text_1 = h.handle(html_data)
            main_doc_list.append(text_1)
            for document_no in enumerate(main_doc_list):
                out_dict.update({filename:main_doc_list[document_no[0]]})

        if file_extension == '.txt':
            text_text = open(doc, 'r')
            text_1 = text_text.read() 
            main_doc_list.append(text_1)
            for document_no in enumerate(main_doc_list):
                out_dict.update({filename:main_doc_list[document_no[0]]})    

    return out_dict

*main_func_call()* will call the *auto_load()*, store the paths in **input_f** that are called in *doc_to_text()* which transforms the files and returns the final dict of textual data.


In [7]:
def main_func_call(path):
    input_f = auto_load(path)
    final_text = doc_to_text(input_f)
    return final_text

**input_folder** stores the path of the input folder.

In [8]:
""" "/content/drive/MyDrive/Colab_Notebooks1/Practice/SSC_GCA" this is my local system path """

input_folder = str(input("Please enter the folder path: "))

Please enter the folder path: /content/drive/MyDrive/Colab_Notebooks1/Practice/SSC_GCA


In [69]:
final_data = main_func_call(input_folder)

**final_data** has a length of 8 documents because the input folder has 8 documents.


In [71]:
len(final_data)

8

###Output file: 
*   final_data ['Artemis_NASA.html'] output shows that the dict has key values pair. The values can be retrived using keys.


In [ ]:
final_data['Artemis_NASA.html']

This is the final output dict that contains key-value pairs for all the documents.

In [ ]:
final_data

In [88]:
filename= '/content/drive/MyDrive/Colab_Notebooks1/Practice/SSC_GCA/pickled_data'
outfile= open(filename, 'wb')

In [89]:
pickle.dump(final_data, outfile)
outfile.close()